In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/Project 0/Project 0/Notebooks"
import sys
sys.path.append(path)
from funcs import *

Mounted at /content/drive


#AP Hybrid Model

In [ ]:
path = path
imageFolder = 'Scanpath Images - AP'
predsDict = {'Subject_Name':[],'y_True':[]}
predsDict['Scanpath_Model_Predictions'] = []
predsDict['Temporal_Model_Predictions'] = []
imageList = ['Scanpath','Temporal']
train, val, test = getTrainTest(path,positiveClass='AP', valSet=True,imageFolder=imageFolder)

for i in range(len(test)):
  for j in imageList:
    imageFolder = f'{j} Images - AP'
    testData = Image2Data('RGB', test[i], path, imageFolder) 
    testX, testy  = testData.X, testData.y
    sumPreds = np.zeros((testy.shape[0],1))
    for r in range(10):
      model = keras.models.load_model(f'{path}/model_{j.lower()}_images_ap_run_{r}_{i}')
      preds = (model.predict(testX)).reshape(testy.shape[0],1)  
      sumPreds += preds
    meanPreds = sumPreds/10
    predsDict[f'{j}_Model_Predictions'].extend(meanPreds)

  predsDict['y_True'].extend(testy)
  predsDict['Subject_Name'].extend(testData.names)

KeyboardInterrupt: ignored

In [ ]:
predsDf = pd.DataFrame.from_dict(predsDict)

In [ ]:
hybrid= {'Test':[],'Temporal_Model_Weight':[],'Scanpath_Model_Weight':[],'AUC':[],
              'Positive_Class_Ratio':[],'Negative_Class_Ratio':[]}

for i in range(len(test)):
  tmpDict = {'Scanpath':[],'Temporal':[],'AUC':[]}
  tmpDf = predsDf[predsDf.Subject_Name.isin(test[i])]
  
  X = tmpDf.drop(columns=['Subject_Name','y_True'])
  y = tmpDf.y_True

  # A loop that finds the optimal weights for each set by maximizing AUC
  weights = np.linspace(0,1,1000)
  for w in weights:
    iw = 1-w
    tmpDict['Scanpath'].append(w)
    tmpDict['Temporal'].append(iw)
    tmpDict['AUC'].append(aucScore(y,(w * tmpDf['Scanpath_Model_Predictions']+iw * tmpDf['Temporal_Model_Predictions']))) 

  maxAUC = pd.DataFrame.from_dict(tmpDict).sort_values(by='AUC',ascending=False).iloc[0,:]
  preds = maxAUC['Scanpath'] * tmpDf['Scanpath_Model_Predictions'] + maxAUC['Temporal'] * tmpDf['Temporal_Model_Predictions']
  
  hybrid['Test'].append(test[i])
  hybrid['Temporal_Model_Weight'].append(maxAUC['Temporal'])
  hybrid['Scanpath_Model_Weight'].append(maxAUC['Scanpath'])
  hybrid['AUC'].append(maxAUC['AUC'])
  hybrid['Positive_Class_Ratio'].append(len(tmpDf[tmpDf.y_True==1])/len(tmpDf))
  hybrid['Negative_Class_Ratio'].append(len(tmpDf[tmpDf.y_True==0])/len(tmpDf))

In [ ]:
apHybrid = pd.DataFrame(ensResults)
apHybrid.to_csv(path+'/AP_Hybrid_Results.csv',index=False)

# Plotting AP Results

In [ ]:
apHybrid = pd.read_csv(path+'/AP_Hybrid_Results.csv')

apScanDf = pd.read_csv(path+'/AP_Mean_Scanpath_Results_AutoEpoch.csv')
apScanDf.columns = pd.MultiIndex.from_arrays([apScanDf.columns, apScanDf.iloc[0].values])
apScanDf = (apScanDf.iloc[1:]).astype(float)


apTempDf = pd.read_csv(path+'/AP_Mean_Temporal_Results_AutoEpoch.csv')
apTempDf.columns = pd.MultiIndex.from_arrays([apTempDf.columns, apTempDf.iloc[0].values])
apTempDf = (apTempDf.iloc[1:]).astype(float)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
x = [f'Fold {i}' for i in range(1,9)]
fig = go.Figure(data=[go.Bar(name='Hybrid Model', x=x, y=apHybrid.AUC,marker_color='black'),
                      go.Bar(name='Facial Scanpath CNN', x=x, y=apScanDf['test_auc']['mean'],marker_color='gray',error_y=dict(type='data',array=(apScanDf['test_auc.1']['std']/2).tolist())),
                      go.Bar(name='ROI Sequence CNN', x=x, y=apTempDf['test_auc']['mean'],marker_color='lightgray',error_y=dict(type='data',array=(apTempDf['test_auc.1']['std']/2).tolist()))
                      ])
                                                          
# Change the bar mode
fig.update_layout(barmode='group',paper_bgcolor='white',plot_bgcolor='white',autosize=False,width=1000,legend=dict(font_size=20))
fig.update_xaxes(title_font_size=25,tickfont=dict(size=20))
fig.update_yaxes(title_font_size=25,tickfont=dict(size=20))
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
meanHybrid = apHybrid.AUC.mean()
meanFS = apScanDf['test_auc']['mean'].mean()
meanRS = apTempDf['test_auc']['mean'].mean()
baseline = 0.61

x = ['Hybrid','Facial Scanpath','ROI Sequence','Baseline']
y = [meanHybrid, meanFS, meanRS, baseline]
colors = ['black','gray','lightgray','darkgray']
fig = go.Figure(data=[go.Bar(x=x, y=y,marker_color=colors)])
                                                          
# Change the bar mode
fig.update_layout(barmode='group',paper_bgcolor='white',plot_bgcolor='white',autosize=False,width=700,height=800,legend=dict(font_size=20))
fig.update_xaxes(tickfont=dict(size=35))
fig.update_yaxes(tickfont=dict(size=35))
fig.show()

# DP Hybrid Model

In [ ]:
path = path
imageFolder = 'Scanpath Images - DP'
predsDict = {'Subject_Name':[],'y_True':[]}
predsDict['Scanpath_Model_Predictions'] = []
predsDict['Temporal_Model_Predictions'] = []
imageList = ['Scanpath','Temporal']
train, val, test = getTrainTest(path,positiveClass='DP', valSet=True,imageFolder=imageFolder)

for i in range(len(test)):
  for j in imageList:
    imageFolder = f'{j} Images - DP'
    testData = Image2Data('RGB', test[i], path, imageFolder) 
    testX, testy  = testData.X, testData.y
    sumPreds = np.zeros((testy.shape[0],1))
    for r in range(10):
      model = keras.models.load_model(f'{path}/model_{j.lower()}_images_dp_run_{r}_13_{i}')
      preds = (model.predict(testX)).reshape(testy.shape[0],1)  
      sumPreds += preds
    meanPreds = sumPreds/10
    predsDict[f'{j}_Model_Predictions'].extend(meanPreds)

  predsDict['y_True'].extend(testy)
  predsDict['Subject_Name'].extend(testData.names)

In [ ]:
predsDf = pd.DataFrame.from_dict(predsDict)

In [ ]:
ensResults = {'Test':[],'Temporal_Model_Weight':[],'Scanpath_Model_Weight':[],'AUC':[],
              'Positive_Class_Ratio':[],'Negative_Class_Ratio':[]}

for i in range(len(test)):
  tmpDict = {'Scanpath':[],'Temporal':[],'AUC':[]}
  tmpDf = predsDf[predsDf.Subject_Name.isin(test[i])]
  
  X = tmpDf.drop(columns=['Subject_Name','y_True'])
  y = tmpDf.y_True

  # A loop that finds the optimal weights for each set by maximizing AUC
  weights = np.linspace(0,1,1000)
  for w in weights:
    iw = 1-w
    tmpDict['Scanpath'].append(w)
    tmpDict['Temporal'].append(iw)
    tmpDict['AUC'].append(aucScore(y,(w * tmpDf['Scanpath_Model_Predictions']+iw * tmpDf['Temporal_Model_Predictions']))) 

  maxAUC = pd.DataFrame.from_dict(tmpDict).sort_values(by='AUC',ascending=False).iloc[0,:]
  preds = maxAUC['Scanpath'] * tmpDf['Scanpath_Model_Predictions'] + maxAUC['Temporal'] * tmpDf['Temporal_Model_Predictions']

  #report = classification_report(y, predLabel, target_names=['negative', 'positive'], output_dict=True)
  ensResults['Test'].append(test[i])
  ensResults['Temporal_Model_Weight'].append(maxAUC['Temporal'])
  ensResults['Scanpath_Model_Weight'].append(maxAUC['Scanpath'])
  ensResults['AUC'].append(maxAUC['AUC'])

  ensResults['Positive_Class_Ratio'].append(len(tmpDf[tmpDf.y_True==1])/len(tmpDf))
  ensResults['Negative_Class_Ratio'].append(len(tmpDf[tmpDf.y_True==0])/len(tmpDf))

In [ ]:
dpHybrid = pd.DataFrame(ensResults)
dpHybrid.to_csv(path+'/DP_Hybrid_Results.csv',index=False)

# Plotting DP Results

In [ ]:
dpHybrid = pd.read_csv(path+'/DP_Hybrid_Results.csv')

dpScanDf = pd.read_csv(path+'/DP_Mean_Scanpath_Results_AutoEpoch.csv')
dpScanDf.columns = pd.MultiIndex.from_arrays([dpScanDf.columns, dpScanDf.iloc[0].values])
dpScanDf = (dpScanDf.iloc[1:]).astype(float)


dpTempDf = pd.read_csv(path+'/DP_Mean_Temporal_Results_AutoEpoch.csv')
dpTempDf.columns = pd.MultiIndex.from_arrays([dpTempDf.columns, dpTempDf.iloc[0].values])
dpTempDf = (dpTempDf.iloc[1:]).astype(float)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
x = [f'fold{i}' for i in range(1,11)]
fig = go.Figure(data=[go.Bar(name='Hybrid Model', x=x, y=dpHybrid.AUC,marker_color='black'),
                      go.Bar(name='Facial Scanpath CNN', x=x, y=dpScanDf['test_auc']['mean'],marker_color='gray',error_y=dict(type='data',array=(dpScanDf['test_auc.1']['std']/2).tolist())),
                      go.Bar(name='ROI Sequence CNN', x=x, y=dpTempDf['test_auc']['mean'],marker_color='lightgray',error_y=dict(type='data',array=(dpTempDf['test_auc.1']['std']/2).tolist()))
                                                          ])

# Change the bar mode
fig.update_layout(barmode='group',paper_bgcolor='white',plot_bgcolor='white',autosize=False,width=1000,legend=dict(font_size=20))
fig.update_xaxes(tickfont=dict(size=20))
fig.update_yaxes(tickfont=dict(size=20))
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
meanHybrid = dpHybrid.AUC.mean()
meanFS = dpScanDf['test_auc']['mean'].mean()
meanRS = dpTempDf['test_auc']['mean'].mean()
baseline = 0.67

x = ['Hybrid','Facial Scanpath','ROI Sequence','Baseline']
y = [meanHybrid, meanFS, meanRS, baseline]
colors = ['black','gray','lightgray','darkgray']
fig = go.Figure(data=[go.Bar(x=x, y=y,marker_color=colors)])
                                                          
# Change the bar mode
fig.update_layout(barmode='group',paper_bgcolor='white',plot_bgcolor='white',autosize=False,width=700,height=800,legend=dict(font_size=20))
fig.update_xaxes(tickfont=dict(size=35))
fig.update_yaxes(tickfont=dict(size=35))
fig.show()